In [3]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### Pré-Processamento

In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    return image

images = []
labels = []
label_to_index = {"direita": 0, "esquerda": 1, "frente": 2}

for label in ["direita", "esquerda", "frente"]:
    for image_path in os.listdir(f"{label}"):
        if image_path.endswith('.jpg'):
            image = preprocess_image(f"{label}/{image_path}")
            images.append(image)
            labels.append(label_to_index[label])

images = np.array(images)
labels = np.array(labels)

labels = to_categorical(labels, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


### Treinar o modelo

In [5]:

model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))



Epoch 1/2
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 297ms/step - accuracy: 0.5393 - loss: 4.1788 - val_accuracy: 0.8646 - val_loss: 0.4191
Epoch 2/2
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 292ms/step - accuracy: 0.8799 - loss: 0.3037 - val_accuracy: 0.9028 - val_loss: 0.3149


### Salvar o modelo no formato Keras

In [6]:
model.save('modelo_direcao.keras')